# Spot Size Calculator

Mark Brown  
Notebook Base Version 1.2

In [3]:
date = "17????"

# General Setup

## Imports and Settings

### Miscellaneous Imports

In [4]:
import numpy as np
from numpy import array as arr
import pandas as pd
import collections
import math as m
from mpl_toolkits.mplot3d import axes3d
from astropy.io import fits

### Scipy

In [5]:
import scipy
from scipy.optimize import curve_fit as fit
import scipy.special as special

### Matplotlib Settings

In [6]:
import matplotlib as mpl
from matplotlib.pyplot import *
### set matplotlib plot defaults :D
%matplotlib inline
# Style controls many default colors in matplotlib plots.
# Change the following if you don't like dark backgrounds. Many other options.
style.use(['dark_background'])
mpl.rcParams['axes.facecolor'] = '#0a0a0a'
# the default cycling of colors in this mode isn't very good.
mpl.rcParams['axes.prop_cycle'] = cycler('color', ['r','c','g','#FFFFFF','y','m','b'])
mpl.rcParams['figure.figsize'] = (18.0, 8.0)
mpl.rcParams['axes.grid'] = True
mpl.rcParams['axes.formatter.useoffset'] = False
mpl.rcParams['grid.alpha'] = 0.3
mpl.rcParams['axes.formatter.limits'] = (0,1)
# jet is awful.
mpl.rcParams['image.cmap'] = 'inferno'
# to see all available options, decomment this line.
#print(mpl.rcParams)
mpl.rcParams['font.size'] = 14

### SymPy

In [7]:
import sympy as sp
sp.init_printing(use_latex=True)
# see the constants section for some constants set in sympy

### Javascript Settings

In [8]:
    %%javascript
    // the above line makes this entire cell run javascript commands.
    // this gets rid of scroll bars on the output by default. It's in javascript because javascript is used 
    // by Jupyter to actually render the notebook display.
    IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

## Constants

### Physics & Math Constants

In [9]:
# all in mks
# planck's constant
h = 6.6260700e-34
# reduced planck's constant
hbar = 1.0545718e-34
# boltzman's constant
k_B = 1.380649e-23
# speed of light (exact)
c = 299792458
# Stephan-Boltzman constant
sigma = 5.6704e-8
# atomic mass unit
amu = 1.6605390e-27
# rubidium 87 mass
Rb87_M = 86.909180527 * amu
# use numpy
pi = np.pi
# gravity constant
g = 9.80665

# linewidths
Rb87_D1Gamma = 36.1e6
Rb87_D2Gamma = 38.11e6

Rb87_I_Sat = 3.576
### should refine... for rubidium 87
# in m
Rb87_D2LineWavelength = 780e-9
# in Hz (1/s)
Rb87_D2LineFrequency = 377.1e12
# etc.
Rb87_D1LineWavelength = 795e-9
Rb87_D1LineFrequency = 384.2e12

### Lab Constants

In [10]:
andorDataRepository = "\\\\andor\\share\\Data and documents\\Data repository\\"
rawDataLoc = date + "\\Raw Data\\"

opBeamDacToVoltageConversionConstants = [8.5, -22.532, -1.9323, -0.35142]
# 7.4 x 7.4 micron mixel size
baslerScoutCcdPixelSize = 7.4e-6
# baslerAceCMosPixelSize = ???
# 16 micron pixels
andorPixelSize = 16e-6

### basler conversion... joules incident per greyscale count.
# number from theory of camera operation
cameraConversion = 117*10**-18 
# number from measurement. I suspect this is a little high because I think I underestimated the attenuation 
# of the signal by the 780nm filter.
# C = 161*10**-18 

### Lab Volatile Constants

Constants that can easily change day to day depending on drifts, etc.

In [11]:
# Calibration of length (in meters) that atoms in the cell move vs pixels that the atoms move.
# this changes when the imaging system for the basler changes. there's a notebook for calculating this. 
baslerMetersPer4x4Pixel = 61.7818944758e-6
baslerMetersPerPixel = baslerMetersPer4x4Pixel / 4

### TODO: normalize these units to my standard units...

# in mW
sidemotPower = 0.75
# in mW
diagonalMPower = 9.3
# in cm
motRadius = 5 * baslerMetersPer4x4Pixel * 100
# in hertz
imagingDetuning = 10*10**6

baslerRawGain = 260

# in cm
axialImagingLensDiameter = 2.54
# in cm
axialImagingLensFocalLength = 10

### Sympy Constants

In [12]:
# Pauli Matrices
X = sigma_x = sp.Matrix([[0,1],[1,0]])
Y = sigma_y = sp.Matrix([[0,-1j],[1j,0]])
Z = sigma_z = sp.Matrix([[1,0],[0,-1]])
# Hadamard
H = hadamard = sp.Matrix([[1,1],[1,-1]])
# Phase Gate
S = phaseGate = sp.Matrix([[1,0],[0,1j]])
# Phase Shift gate
def phaseShiftGate(phi):
    return sp.Matrix([[1,0],[[0,sp.exp(1j*phi)]]])


## Functions

### Random Useful Functions

In [13]:
def round_sig(x, sig=3):
    return round(x, sig-int(m.floor(m.log10(abs(x)+np.finfo(float).eps)))-1)

### Generic functions for fits

In [14]:
def quadratic(x,a,b,x0):
    # This assumes downward facing. Best to write another function for upward facing if need be, I think.
    if a < 0:
        return 10**10
    if b > 0:
        return 10**10
    return a + b*(x-x0)**2


def gaussian(x, A1, x01, sig1, offset):
    if (offset < 0):
        return 10**10
    return offset + A1 * np.exp(-(x-x01)**2/(2*sig1**2))


def doubleGaussian(x, A1, x01, sig1, A2, x02, sig2, offset):
    if (A1 < 0 or A2 < 0):
        # Penalize negative fits.
        return 10**10
    if (offset < 0):
        return 10**10
    return offset + A1 * np.exp(-(x-x01)**2/(2*sig1**2)) + A2 * np.exp(-(x-x02)**2/(2*sig2**2))


def tripleGaussian(x, A1, x01, sig1, A2, x02, sig2, A3, x03, sig3, offset ):
    if (A1 < 0 or A2 < 0 or A3 < 0):
        # Penalize negative fits.
        return 10**10
    if (offset < 0):
        return 10**10
    return (offset + A1 * np.exp(-(x-x01)**2/(2*sig1**2)) + A2 * np.exp(-(x-x02)**2/(2*sig2**2)) 
            + A3 * np.exp(-(x-x03)**2/(2*sig3**2)))

# Stolen from "http://stackoverflow.com/questions/21566379/fitting-a-2d-gaussian
#-function-using-scipy-optimize-curve-fit-valueerror-and-m"
def gaussian_2D(coordinates, amplitude, xo, yo, sigma_x, sigma_y, theta, offset):
    # limit the angle to a small range to prevent unncecessary flips of the axes. The 2D gaussian has two axes of symmetry,
    # so only a quarter of the 2pi is needed.
    if theta > pi/4 or theta < -pi/4:
        return 10e10
    x = coordinates[0]
    y = coordinates[1]
    xo = float(xo)
    yo = float(yo)    
    a = (np.cos(theta)**2)/(2*sigma_x**2) + (np.sin(theta)**2)/(2*sigma_y**2)
    b = -(np.sin(2*theta))/(4*sigma_x**2) + (np.sin(2*theta))/(4*sigma_y**2)
    c = (np.sin(theta)**2)/(2*sigma_x**2) + (np.cos(theta)**2)/(2*sigma_y**2)
    g = offset + amplitude*np.exp( - (a*((x-xo)**2) + 2*b*(x-xo)*(y-yo) + c*((y-yo)**2)))
    return g.ravel()

def decayingCos(x, A, tau, f, phi, offset):
    # Just for sanity. Keep some numbers positive.
    if (A < 0):
        return x * 10**10
    if (phi < 0):
        return x * 10**10
    if (offset < 0):
        return x * 10**10
    # no growing fits.
    if (tau > 0):
        return x * 10**10
    return offset + (1 - A/2 * np.exp(-x/tau) * np.cos(2 * np.pi * f * x + phi))

def sincSquared(x, A, center, scale, offset):
    if (offset < 0):
        return x * 10**10
    if (A < 0):
        return x * 10**10
    return (A * np.sinc((x - center)/scale)**2 + offset)


def lorentzian(x, A, center, width, offset):
    if (offset < 0):
        return x * 10**10
    if (A < 0):
        return x * 10**10
    return (A /((x - center)**2 + (width/2)**2))


def poissonian(x, k, weight):    
    """
    This function calculates p_k{x} = weight * e^(-k) * k^x / x!.
    :param x: argument of the poissonian
    :param k: order or (approximate) mean of the poissonian.
    :param weight: a weight factor, related to the maximum data this is supposed to be fitted to, but typically over-
    weighted for the purposes of this function.
    :return: the poissonian evaluated at x given the parametes.
    """
    import numpy as np
    term = 1
    # calculate the term k^x / x!. Can't do this directly, x! is too large.
    for n in range(0, int(x)):
        term *= k / (x - n)
    return np.exp(-k) * term * weight

# Main Function Definition

## Some Derivation

$$
\frac{2 J_1\{x\}}{x}
$$

In [19]:
def jinc(x):
    if x==0:
        return 1.0
    return 2 * special.jv(1,x)/x


def airydisk(x):
    # airy = jinc^2
    return jinc(x)**2


def airy_2(x):
    # to find 1/e^2 point
    # (jinc - 1/e^2 jinc)**2 = 0
    return (airydisk(x) - 1/np.e**2*airydisk(0))**2

In [20]:
zero = scipy.optimize.fmin(airydisk, 1)
eSquared = scipy.optimize.fmin(airy_2, 1)
print('zero:', zero)
print('esquared:', eSquared)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 20
         Function evaluations: 41
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 18
         Function evaluations: 36
zero: [ 3.83173828]
esquared: [ 2.58388672]


1/e^2 point = 2.583 / 3.8317 * zero point

## spotSize()

$$
w_f = \frac{\lambda f}{\pi w_0}
$$

In [21]:
def spotSize(waist, aperture, focalLength, wavelength=850e-9, type='gaussian'):
    """
    Calculate the expected spot size
    
    *********** ***********
    *** Inputs:
    
    :waist: Waist (2 sigma) of the input beam. Ignored if type is 'airy'.
    :aperture: Aperture *radius* of the input beam. Ignored if type is 'gaussian'.
    :focalLength: effective focal length of the focusing lens, in m.
    :wavelength: the wavelength of the light being focused.
    :type: (Expects one of 'gaussian', 'airy', 'convolution') Specifies the type of calculation. 'convolution'
        is not currently supported, but could be constructed out of the calcualtions below. It Would be fairly 
        computationally expensive. 
        - If 'gaussian', beam is approximated to have not been apertured at all and the 1/e^2 waist (radius) is returned,
        - If 'airy', beam is approximated as a plane wave incident on an aperture and the 1/e^2 radius is returned.
        - If 'airy-zero', beam is approximated as a plane wave incident on an aperture and the first zero is returned.
    """
    
    if type=='gaussian':
        return wavelength * focalLength / (pi * waist)
    elif type=='airy':
        # this comes from knowing the location of the first null analytically and multiplying by the ratio of the distance to the 
        # first null of the bessel function and the 1/e^2 radius of the airy disk; I calculated these myself.
        # See the original notebook for this function for how this is calculated, it's just some scipy optimization.
        return wavelength * focalLength / (2 * aperture) * (2.58388672 / 3.83173828)
    elif type=='airy-zero':
        return wavelength * focalLength / (2 * aperture)
    elif type=='convolution':
        raise ValueError('"type"=="convolution" is not yet supported..')
    else:
        raise ValueError('Argument value for parameter "type" is not valid.')    

# Convolution Work

## Definitions

In [ ]:
# using the standard definition for a field gaussian, exp(-x^2/w^2)
def radialGaussian(x, y, A, waist, offset):
    r = np.sqrt(x**2 + y**2)
    return A * np.exp(-r**2 / waist**2) + offset
def circ(x,y,R):
    r = np.sqrt(x**2 + y**2)
    if r > R:
        return 0
    else:
        return 1

In [ ]:
# the fourier transform of a circle, or a "jinc" function.
def circTransformed(fx, fy, D):
    return D * special.jv(1,2*pi*D*np.sqrt(fx**2 + fy**2)) / np.sqrt(fx**2 + fy**2)

def gaussianTransformed(fx, fy, w0):
    return w0**2 * np.exp(-np.pi**2 * w0**2 * (fx**2 + fy**2))

def convolutionIntegrand(xi, eta, fx, fy, D, w):
    x = circTransformed(xi,eta, D) * gaussianTransformed(fx - xi, fy - eta, w)
    return x

## Inputs

In [ ]:
apertureRadius = 20e-3
inputWaist = 40e-3
wavelength = 850e-9
focalLength = 22e-3

## Calculations

In [ ]:
x = y = np.linspace(-3e-2,3e-2,200)
X, Y = np.meshgrid(x,y)
inputField = np.zeros(X.shape)
f_gaussian = np.zeros(X.shape)
f_circle = np.zeros(X.shape)
for x in range(X.shape[0]):
    for y in range(X.shape[1]):
        xi = X[x,y]
        yi = Y[x,y]
        inputField[x,y] = radialGaussian(xi,yi, 1, inputWaist, 0) * circ(xi, yi, apertureRadius)
for x in range(X.shape[0]):
    for y in range(X.shape[1]):
        # it's small so use small values
        fx = X[x,y]*2e-5 / (wavelength*focalLength)
        fy = Y[x,y]*2e-5 / (wavelength*focalLength)
        f_gaussian[x,y] = gaussianTransformed(fx, fy, inputWaist)
for x in range(X.shape[0]):
    for y in range(X.shape[1]):
        # it's small so use small values
        fx = X[x,y]*2e-5 / (wavelength*focalLength)
        fy = Y[x,y]*2e-5 / (wavelength*focalLength)
        f_circle[x,y] = circTransformed(fx, fy, 2*apertureRadius)
        
        

In [ ]:
fieldAtFocus = np.zeros(X.shape)
count = 0
bound = max((X*2e-5 / (wavelength*focalLength))[0,:])*2
for x in range(X.shape[1]):
    for y in range(X.shape[0]):
        count += 1
        if count % 200 == 0:
            print(count)
        fxi = X[x,y]*2e-5 / (wavelength * focalLength)
        fyi = Y[x,y]*2e-5 / (wavelength * focalLength)
        #def convolutionIntegrand(xi, eta, fx, fy, D, w):
        
        lims = [[-bound, bound*1.1], [-bound, bound*1.1]]
        fieldAtFocus[x,y] = scipy.integrate.nquad(convolutionIntegrand, lims, args=[fxi,fyi,2*apertureRadius,inputWaist], opts={"limit":100})[0]

## Results

In [ ]:
figure(figsize=(13,10))
subplot(221)
pcolormesh(X, Y, inputField)
colorbar()
title('input field');

subplot(222)
pcolormesh(X, Y, fieldAtFocus)
colorbar()
title('input field at focus');

subplot(223)
pcolormesh(X*1e-4, Y*1e-4, f_gaussian)
colorbar()
title('gaussian field at focus');

subplot(224)
pcolormesh(X*1e-4, Y*1e-4, f_circle)
colorbar()
title('circle field at focus');


### 1D convolution test

Appears to be working fine...

In [ ]:
def oneDCov(fxi, D, w0, xi):
    return np.exp(-pi**2*w0**2*(fxi-xi)**2) * np.sin(pi*D*xi)/(pi*xi)

In [ ]:
x = np.linspace(-6e-2, 6e-2, 1000)
inputWaist = 2e-2
oneDField = np.zeros(len(x))
for xi in range(len(x)):
    fxi = x[xi]*2e-5 / (wavelength * focalLength)
    convInt = lambda a: oneDCov(fxi, 2*apertureRadius, inputWaist, a)
    lims = [[-.1, 0.2]]
    oneDField[xi] = scipy.integrate.nquad(convInt, lims)[0]
plot(x,oneDField)
print(max(oneDField))
print(2*apertureRadius)
data = max(oneDField) * np.sin(2*apertureRadius*pi*x*2e-5/(wavelength*focalLength)) / (2*apertureRadius*pi*x*2e-5/(wavelength*focalLength))
plot(x, data)

# Work Space